In [1]:
!pip install -q transformers datasets evaluate scikit-learn pandas torch matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

df = pd.read_csv("intent_data_extended.csv")
df.head()

,text,label
0,I'm overjoyed!,happy
1,Life couldn't be better,happy
2,This is so exciting...,happy
3,This is absolutely amazing!!,happy
4,Everything is awesome today,happy


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df["label"])
num_labels = len(label_encoder.classes_)

In [4]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label_id"],
    random_state=42
)

In [5]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[["text", "label_id"]])
val_dataset = Dataset.from_pandas(val_df[["text", "label_id"]])

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("label_id", "labels")
val_dataset = val_dataset.rename_column("label_id", "labels")

train_dataset.set_format("torch")
val_dataset.set_format("torch")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [7]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [9]:
import evaluate, numpy as np

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

/tmp/ipython-input-3967590506.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 21btrca065 (21btrca065-jain-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.182920,0.800000,0.759991
2,No log,0.756131,0.900000,0.893721
3,No log,0.498458,1.000000,1.000000
4,No log,0.357730,1.000000,1.000000
5,No log,0.304702,1.000000,1.000000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument i

{'eval_loss': 0.4984577000141144,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 12.9922,
 'eval_samples_per_second': 4.618,
 'eval_steps_per_second': 0.308,
 'epoch': 5.0}

In [11]:
model.save_pretrained("bert_emotion_model")
tokenizer.save_pretrained("bert_emotion_model")

import joblib
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [12]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification
import joblib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained("bert_emotion_model")
model = BertForSequenceClassification.from_pretrained("bert_emotion_model")
label_encoder = joblib.load("label_encoder.pkl")

model.to(device)
model.eval()

def predict_emotion_with_confidence(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)

    conf, pred = torch.max(probs, dim=1)
    emotion = label_encoder.inverse_transform([pred.item()])[0]

    return emotion, conf.item()

def chatbot_response(text):
    emotion, confidence = predict_emotion_with_confidence(text)

    responses = {
        "happy": "I'm glad you're feeling happy 😊",
        "sad": "I'm here for you 😔",
        "angry": "Let’s take a deep breath 😠",
        "neutral": "Tell me more."
    }

    return responses.get(emotion, "I understand."), confidence

In [13]:
print("🤖 Chatbot is running (type 'bye' to exit)\n")

while True:
    user_input = input("You: ")

    if user_input.lower() in ["bye", "exit", "quit"]:
        print("Chatbot: Take care ❤️")
        break

    reply, conf = chatbot_response(user_input)
    print(f"Chatbot: {reply} (confidence: {conf:.2f})")

🤖 Chatbot is running (type 'bye' to exit)

You: I can't tolerate this.
Chatbot: Let’s take a deep breath 😠 (confidence: 0.51)
You: I'm feeling upset.
Chatbot: I'm here for you 😔 (confidence: 0.59)
You: You are looking very happy today.
Chatbot: I'm glad you're feeling happy 😊 (confidence: 0.47)
You: Are you going there?
Chatbot: Tell me more. (confidence: 0.37)
You: bye
Chatbot: Take care ❤️
